In [1]:
import matplotlib.pyplot as plt
import numpy as np
import importlib
import Scsv


In [2]:
# %matplotlib widget

In [3]:
%matplotlib

Using matplotlib backend: agg


In [4]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import sys
from operator import add

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StopWordsRemover

In [5]:
# MASTER= "spark://virtual01-virtualbox:7077"
MASTER= "local[*]"

spark = SparkSession.builder\
    .appName("MyProj")\
    .master(MASTER)\
    .config("spark.sql.repl.eagerEval.enabled", True)\
    .getOrCreate()

fname="hdfs://zuk:9000/dataset/twitter/Tweets.csv"

In [6]:
df = spark.read.option("header",True).csv(fname)\
    .select("airline_sentiment","text")

rter = RegexTokenizer(inputCol="text", outputCol="token", pattern="[\\W]+", minTokenLength=3)
tokenized = rter.transform(df.na.fill(' '))

In [7]:
more_stopwords=["virginamerica","https","ual","nyc"]
more_stopwords.extend(StopWordsRemover().getStopWords())

remover = StopWordsRemover(inputCol="token", outputCol="rtoken").setStopWords(more_stopwords)
removed = remover.transform(tokenized)

removed.count()

14837

In [8]:
removed.select("airline_sentiment","rtoken").limit(10)

airline_sentiment,rtoken
neutral,"[dhepburn, said]"
positive,"[plus, added, com..."
neutral,"[didn, today, mus..."
negative,"[really, aggressi..."
negative,"[really, big, bad..."
negative,"[seriously, pay, ..."
,[]
positive,"[yes, nearly, eve..."
neutral,"[really, missed, ..."
positive,"[well, didn]"


In [9]:
oname = fname.replace("/", "").replace(":", "")
# !transform data

word2Vec = Word2Vec(vectorSize=32, minCount=1,
                    inputCol="rtoken", outputCol="vector")

model = word2Vec.fit(removed)


result = model.transform(removed)

In [10]:
samples = 100

wds1 = result.select("rtoken")\
    .filter("airline_sentiment = \"positive\" OR airline_sentiment = \"neutral\"")\
    .head(samples)
vecs1 = result.select("vector")\
    .filter("airline_sentiment = \"positive\" OR airline_sentiment = \"neutral\"")\
    .head(samples)

wds2 = result.select("rtoken")\
    .filter(result.airline_sentiment =="negative")\
    .head(samples)
vecs2 = result.select("vector")\
    .filter(result.airline_sentiment =="negative")\
    .head(samples)

cnt1 = len(vecs1)
cnt2 = len(vecs2)

In [11]:
display(wds1[0:5])
display(wds2[0:5])

[Row(rtoken=['dhepburn', 'said']),
 Row(rtoken=['plus', 'added', 'commercials', 'experience', 'tacky']),
 Row(rtoken=['didn', 'today', 'must', 'mean', 'need', 'take', 'another', 'trip']),
 Row(rtoken=['yes', 'nearly', 'every', 'time', 'fly', 'ear', 'worm', 'won', 'away']),
 Row(rtoken=['really', 'missed', 'prime', 'opportunity', 'men', 'without', 'hats', 'parody', 'mwpg7grezp'])]

[Row(rtoken=['really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guests', 'faces', 'amp', 'little', 'recourse']),
 Row(rtoken=['really', 'big', 'bad', 'thing']),
 Row(rtoken=['seriously', 'pay', 'flight', 'seats', 'didn', 'playing']),
 Row(rtoken=['sfo', 'pdx', 'schedule', 'still', 'mia']),
 Row(rtoken=['flew', 'sfo', 'last', 'week', 'couldn', 'fully', 'sit', 'seat', 'due', 'two', 'large', 'gentleman', 'either', 'side', 'help'])]

In [12]:
model.findSynonyms("rain",10)


word,similarity
southwestverity,0.8988014459609985
papers,0.8939696550369263
sammi_jon3s,0.8735257983207703
released,0.8612871766090393
100,0.860431432723999
premier,0.8581547141075134
degree,0.8556652069091797
utah,0.8522842526435852
mention,0.8516773581504822
husband,0.8511340022087097


In [13]:
result.printSchema()
result.count()

root
 |-- airline_sentiment: string (nullable = false)
 |-- text: string (nullable = false)
 |-- token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rtoken: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vector: vector (nullable = true)



14837

In [14]:
from pyspark.sql.functions import col
df1 = df.filter("airline_sentiment = \"positive\" OR airline_sentiment = \"neutral\" OR airline_sentiment = \"negative\"")
# unknown sentiment values are removed

df1.groupBy("airline_sentiment")\
    .count()\
    .orderBy(col("count").desc())\
    .show()

+-----------------+-----+
|airline_sentiment|count|
+-----------------+-----+
|         negative| 9178|
|          neutral| 3099|
|         positive| 2363|
+-----------------+-----+



In [15]:
# pipelined, very simplified

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

df2 =  df1.na.fill("")
label_stringIdx = StringIndexer(inputCol="airline_sentiment", outputCol="label")

pipeline=Pipeline(stages = [rter, remover, word2Vec,label_stringIdx])

pipelineFit = pipeline.fit(df2)

df3 = pipelineFit.transform(df2)
display(df3)

airline_sentiment,text,token,rtoken,vector,label
neutral,@VirginAmerica Wh...,"[virginamerica, w...","[dhepburn, said]",[0.04504764056764...,1.0
positive,@VirginAmerica pl...,"[virginamerica, p...","[plus, added, com...",[0.04508242197334...,2.0
neutral,@VirginAmerica I ...,"[virginamerica, d...","[didn, today, mus...",[-0.0058883361052...,1.0
negative,"""@VirginAmerica i...","[virginamerica, r...","[really, aggressi...",[0.01358438511379...,0.0
negative,@VirginAmerica an...,"[virginamerica, a...","[really, big, bad...",[0.07137722894549...,0.0
negative,@VirginAmerica se...,"[virginamerica, s...","[seriously, pay, ...",[-0.0400419132784...,0.0
positive,@VirginAmerica ye...,"[virginamerica, y...","[yes, nearly, eve...",[0.03987884793120...,2.0
neutral,@VirginAmerica Re...,"[virginamerica, r...","[really, missed, ...",[0.00365064489758...,1.0
positive,@virginamerica We...,"[virginamerica, w...","[well, didn]",[0.03906816244125...,2.0
positive,@VirginAmerica it...,"[virginamerica, w...","[amazing, arrived...",[0.09566467329859...,2.0


In [24]:
# training (multi-class-text-classification-with-pyspark)
# ref: https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35

(trainingDf, testDf) = df3.randomSplit([0.75,0.25], seed=100)

print(f"train = {trainingDf.count()}, test count= {testDf.count()}")

train = 11041, test count= 3599


In [25]:
# lr classifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0,
                       featuresCol="vector")
lrModel=lr.fit(trainingDf)

predictions = lrModel.transform(testDf)



predictions.filter(predictions['prediction'] == 0) \
    .orderBy("probability", ascending=False) \
    .limit( 10)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6196671238375006

In [26]:
# random forest
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="vector", \
                            numTrees = 100)# Train model with Training Data
rfModel = rf.fit(trainingDf)

predictions = rfModel.transform(testDf)

predictions.filter(predictions['prediction'] == 0) \
    .orderBy("probability", ascending=False) \
    .limit(10)


evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.6491527920360859

In [ ]:
# perceptron classifier
# ref: https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
#layers = [4, 5, 4, 3]
layers = [32 ,16, 8, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234, featuresCol="vector")

# train the model
pmodel = trainer.fit(trainingDf)

# compute accuracy on the test set
presult = pmodel.transform(testDf)
predictionAndLabels = presult.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))